# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [5]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [3]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [6]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [07:21<00:00, 88.34s/it]


In [7]:
len(deals)

50

In [8]:
deals[44].describe()

"Title: 9-Compartment Portable Wardrobe for $58 + free shipping\nDetails: You'd pay around $95 for very similar models today at Amazon. You can get this price in Grey (pictured) or Blue. Shipping from a local US warehouse is also free. Buy Now at AliExpress\nFeatures: \nURL: https://www.dealnews.com/9-Compartment-Portable-Wardrobe-for-58-free-shipping/21777229.html?iref=rss-c196"

In [9]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [10]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [11]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Unlocked Apple iPhone 16 Pro Max 256GB Smartphone for $890 + free shipping
Details: You'd pay over $900 for most refurbs at Amazon. This one is in "Excellent" condition and comes with a 1-year Allstate warranty. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-16-Pro-Max-256-GB-Smartphone/489845.html?iref=rss-c142


In [12]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [13]:
result = get_recommendations()

In [14]:
len(result.deals)

5

In [15]:
result.deals[1]

Deal(product_description="The Unlocked Apple iPhone 15 offers a sleek design with a powerful A15 Bionic chip for exceptional performance. With 128GB of storage, this smartphone balances performance and capacity, making it suitable for everyday use. This refurbished model is backed by a 1-year warranty, assuring buyers of its quality and reliability. It's a great choice for anyone looking to upgrade their mobile device.", price=415.0, url='https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-15-128-GB-Smartphone/482155.html?iref=rss-c142')

In [16]:
from agents.scanner_agent import ScannerAgent

In [17]:
agent = ScannerAgent()
result = agent.scan()

In [18]:
result

DealSelection(deals=[Deal(product_description="The Refurbished Unlocked Apple iPhone 16 Pro Max features a spacious 256GB storage capacity, making it ideal for anyone who needs ample space for apps, photos, and videos. This smartphone boasts an 'Excellent' condition rating, ensuring a premium feel and functionality. The device includes a 1-year Allstate warranty, providing peace of mind. With its stunning display and powerful performance, this iPhone is a standout choice for tech enthusiasts.", price=890.0, url='https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-16-Pro-Max-256-GB-Smartphone/489845.html?iref=rss-c142'), Deal(product_description='Experience the advanced technology of the Refurbished Unlocked Apple iPhone 15, featuring 128GB of internal storage. This smartphone has been well-reviewed for its sleek design and excellent camera capabilities. It comes with a 1-year Allstate warranty, ensuring reliability and support. Ideal for users looking for a top-tier phone wit